# Adding measurements to Frames

This notebook will outline the general process to add measurement data to frames to allow it to be sent over the network to clients.
Later, it will also cover the means by which we can add automatic measurement calculations and attach these to our sent frames.

## First, create the server/client pair

In [1]:
from nanover.app import NanoverImdApplication
from nanover.websocket import NanoverImdClient
from nanover.websocket.client.app_client import get_websocket_address_from_app_server

# Create server/client pair.
server = NanoverImdApplication.basic_server(port=0)
client = NanoverImdClient.from_url(get_websocket_address_from_app_server(server))

First we will need to load a simulation we want to visualise and compute metrics for.

In [2]:
import MDAnalysis as mda

# Use MDAnalysis to create a nanover simulation to run.
simulation = mda.Universe("../mdanalysis/serotonine_receptor.pdb")

For this tutorial we will compute the distance between a couple of atoms and the radius of gyration of the protein. Then store this information in their relevant NanoVer implementations.
NanoVer provides implementations for 4 measurement types:
- `Scalar`: any scalar metric not relating to geometric information.
- `Distance`: for distances between pairs of atoms.
- `Angle`: for angles between any triplet of atoms.
- `Dihedral`: for torsions between and quadruple of atoms.

In [3]:
import numpy as np
from openmm.unit import angstrom
from nanover.trajectory.measure import *  # OK to wildcard import here.

rad_gyr = Scalar(
    "backbone_rg", simulation.select_atoms("protein and backbone").radius_of_gyration(),
    unit=angstrom
)

atom1, atom2 = simulation.select_atoms("(resnum 75 or resnum 100) and name CA")
distance = Distance(
    "d1",
    atom1.index,
    atom2.index,
    np.linalg.norm(atom1.position - atom2.position),  # Compute distance
    unit=angstrom
)

Next we will create a `MeasureCollection` object to store all of the desired measurements and then add these measurements to a frame containing also containing the structure of the protein.

In [4]:
from nanover.trajectory.measure_collections import MeasureCollection

collection = MeasureCollection(scalars=[rad_gyr], distances=[distance])

collection

MeasureCollection containing:
	1 Scalar; <(Scalar) backbone_rg: 28.960801778835663 A>
	1 Distance; <(Distance) d1: 10.694733619689941 A>

In [5]:
from nanover.mdanalysis import mdanalysis_to_frame_data

frame = mdanalysis_to_frame_data(simulation, topology=True, positions=True)
collection.add_to_frame(frame)

server.frame_publisher.send_frame(frame)

Now we can verify that the sent data does, in fact, contain our desired metrics and can visualise the PDB.

In [6]:
client_measures = MeasureCollection.from_frame(client.current_frame)
client_measures

MeasureCollection containing:
	1 Scalar; <(Scalar) backbone_rg: 28.960801778835663 A>
	1 Distance; <(Distance) d1: 10.694733619689941 A>

## Managing (timeresolved) sets of data.

In some instances you may be working with a pre-computed simulation and it may be simpler to calculate the measurement data all at once, rather than for each frame.
In this case, we can use a `MultiMeasure` object to handle this information.

In [7]:
# Load a trajectory.
trajectory = mda.Universe("../mdanalysis/files/3TI6_ose_wt.pdb", "../mdanalysis/files/ose_wt.dcd")

# Compute the rmsd with respect to first frame.
from MDAnalysis.analysis.rms import RMSD
rmsd_runner = RMSD(trajectory, select="backbone").run()
# Grab just the rmsd values from run results.
rmsd = rmsd_runner.results.rmsd[:, 2]


/home/tn15550/miniforge3/envs/nanover-dev/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


Now we have computed the RMSD for the simulation at each timepoint we can create a `MultiMeasure` object, and iterate over both it and the trajetory to send both bits of information simulatenously.

In [ ]:
# Clear existing collection so only sending over new RMSD values.
collection.clear()

rmsd_measurements = MultiMeasure(Scalar("rmsd", 0, "\u212B"), rmsd)

In [ ]:
from nanover.trajectory import FrameData
from nanover.mdanalysis import mdanalysis_to_frame_data
import time

first_frame = mdanalysis_to_frame_data(trajectory, topology=True)
collection.update(Scalar("rmsd", 0, "\u212B"))
collection.add_to_frame(first_frame)
server.frame_publisher.send_frame(first_frame)

for timepoint, measure in zip(trajectory.trajectory, rmsd_measurements):
    collection.update(measure)

    # Create new frame and add measurement to it.
    frame = FrameData()
    frame.particle_positions = timepoint.positions * 0.1  # Need to convert to nm.
    collection.add_to_frame(frame)

    # Now send frame to client.
    server.frame_publisher.send_frame(frame)
    time.sleep(0.1)  # Slightly delay sending so no info is lost.

In [ ]:
# Client should recieve 25 additional frames all having RMSD values except the first.
client_rmsd = []

for frame in client.frames:
    client_measures = MeasureCollection.from_frame(frame)
    if (measure := client_measures.get_measure("rmsd")) is None:
        continue

    client_rmsd.append(measure)

In [11]:
client_rmsd

[Scalar(name='rmsd', value=1.0969316935571127e-06, unit='Å'),
 Scalar(name='rmsd', value=0.2902518157988404, unit='Å'),
 Scalar(name='rmsd', value=0.4228089433973922, unit='Å'),
 Scalar(name='rmsd', value=0.5033739643120646, unit='Å'),
 Scalar(name='rmsd', value=0.5751169377215607, unit='Å'),
 Scalar(name='rmsd', value=0.6423662101619956, unit='Å'),
 Scalar(name='rmsd', value=0.7059109111118106, unit='Å'),
 Scalar(name='rmsd', value=0.7501970309555404, unit='Å'),
 Scalar(name='rmsd', value=0.7948926818502197, unit='Å'),
 Scalar(name='rmsd', value=0.8413591764815591, unit='Å'),
 Scalar(name='rmsd', value=0.8849450705820308, unit='Å'),
 Scalar(name='rmsd', value=0.9307343888330446, unit='Å'),
 Scalar(name='rmsd', value=0.9671896897540304, unit='Å'),
 Scalar(name='rmsd', value=0.9964474436877444, unit='Å'),
 Scalar(name='rmsd', value=1.0270163390396856, unit='Å'),
 Scalar(name='rmsd', value=1.0687301081360618, unit='Å'),
 Scalar(name='rmsd', value=1.1047092124922544, unit='Å'),
 Scalar(na

## Cleanup

In [12]:
# Close the open server/client ports.
server.close()
client.close()